# JijModeling 1.12.2 Release Notes

## Feature Enhancements

### Error message with source code location

Now the error messages in `Interpreter.eval_*` reports source code location and code fragment when available.
Python provides detailed error span information only in Python 3.11 and later, it is recommended to use Python >= 3.11 to get useful message.

The following is the example of error message with detailed source code locations, which is available Python 3.11+:

In [1]:
import jijmodeling as jm

D = jm.Placeholder("D", dtype=jm.DataType.INTEGER)
T = jm.Placeholder("T", dtype=jm.DataType.INTEGER)
N = jm.Placeholder("N", dtype=jm.DataType.INTEGER)
H = jm.Placeholder("H", shape=(D, T, N))
R = jm.Placeholder("R", shape=(D, N))

x = jm.BinaryVar("x", shape=(D, T, N))
d = jm.Element("d", belong_to=(0, D))
t = jm.Element("t", belong_to=(0, T))
i = jm.Element("i", belong_to=(0, N))

problem = jm.Problem(name="shift_optimization")
problem += jm.Constraint("night_shift", R[d+1, i] <= x[d, T-1, i], forall=[d, i])
problem += jm.sum([d, t, i], x[d, t, i])

problem

In [3]:
num_days = 7
num_times = 3
num_people = 5

data = problem.generate_random_dataset(
    options={
        "D": {"value": num_days},
        "T": {"value": num_times},
        
        "N": {"value": num_people},
    }
)
interp = jm.Interpreter(data)

try:
    interp.eval_problem(problem)
except Exception as e:
    print(e)

Traceback (most recent last):
    while evaluating Problem `shift_optimization',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 14, col 11-48
    while evaluating constraint: `night_shift',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 12-82
    while evaluating expression `R[d + 1, i] - x[d, T - 1, i]',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 41-66
    while evaluating expression `R[d + 1, i]',
        defeind at File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 41-50

File "/var/folders/mg/mg6st30d18s7pxjjrk6pkxym0000gn/T/ipykernel_49615/2988517098.py", line 15, col 41-50:

    15  |  problem += jm.Constraint("night_shift", R[d+1, i] <= x[d, T-1, i], forall=[d, i])
                                                   ^^^^^^^

The same code generates the following message with Python <3.11, which contains linewise information only:

## Other Changes

- Now JijModeling provides ABI3 wheels and supports (non-free-threaded) Python 3.13!